# Deploy with Docker 

The simplest way to deploy a Vespa app.

Install `pyvespa` and make sure Docker is running with minimum 4G:

In [2]:
!docker info | grep "Total Memory"

 Total Memory: 11.7GiB


## Deploy application package created with pyvespa

This section assumes you have an [ApplicationPackage](reference-api.rst#vespa.package.ApplicationPackage) 
instance assigned to `app_package` containing your app desired configuration.
For the purpose of this demonstration we are going to use a minimal (and useless) application package:

In [1]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name="sampleapp")

We can locally deploy our `app_package` using Docker without leaving the notebook, by creating an instance of [VespaDocker](reference-api.rst#vespa.deployment.VespaDocker), as shown below:

In [2]:
import os
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker(port=8080)
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for application status.
Waiting for application status.
Finished deployment.


`app` now holds a [Vespa](reference-api.rst#vespa.application.Vespa) instance, which we are going to use to interact with our application. Congratulations, you now have a Vespa application up and running.

In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()

## Learn Vespa by looking at underlying config files

It is important to know that `pyvespa` simply provides a convenient API
to define Vespa application packages from python.
Going through those files is a nice way to start learning about Vespa syntax.

Use [to_files](reference-api.rst#vespa.package.ApplicationPackage.to_files)
to export the application package package created with `pyvespa` from code to files:

In [3]:
from pathlib import Path
Path("mydir").mkdir(parents=True, exist_ok=True)
app_package.to_files("mydir")

Inspect the exported files:

In [ ]:
!find mydir

It is also possible to export the application package as a zipped file
using [to_zipfile](reference-api.rst#vespa.package.ApplicationPackage.to_zipfile).
The zipfile can later be deployed with pyvespa or the [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html):

In [ ]:
app_package.to_zipfile("myzip.zip")

Remove the files after use:

In [ ]:
from shutil import rmtree
rmtree("mydir", ignore_errors=True)
rmtree("myzip.zip", ignore_errors=True)

## Deploy application package from config files

The `pyvespa` API provides a subset of the functionality available in Vespa. The reason is that `pyvespa` is meant to be used as an experimentation tool for Information Retrieval (IR) and not for building production-ready applications. So, the python API expands based on the needs to replicate common use cases that often require IR experimentation.

If the application requires functionality or fine-tuning not available in `pyvespa`, simply build it directly using Vespa configuration files as shown in [many examples](https://docs.vespa.ai/en/getting-started.html) on Vespa docs. But even in this case, one can still get value out of `pyvespa` by deploying it from python based on the Vespa configuration files stored on disk.

Clone and deploy the news search app covered in this [Vespa tutorial](https://docs.vespa.ai/en/tutorials/news-3-searching.html):

In [4]:
!git clone --depth 1 https://github.com/vespa-engine/sample-apps.git sample-apps

Cloning into 'sample-apps'...
remote: Enumerating objects: 1138, done.
remote: Counting objects: 100% (1138/1138), done.
remote: Compressing objects: 100% (754/754), done.
remote: Total 1138 (delta 189), reused 889 (delta 109), pack-reused 0
Receiving objects: 100% (1138/1138), 18.94 MiB | 9.38 MiB/s, done.
Resolving deltas: 100% (189/189), done.


The Vespa configuration files of the news search app are stored in `sample-apps/news/app-3-searching/`:

In [5]:
!tree sample-apps/news/app-3-searching/

sample-apps/news/app-3-searching/
├── schemas
│   └── news.sd
└── services.xml

1 directory, 2 files


Deploy to a Docker container from disk:

In [6]:
import os
from vespa.deployment import VespaDocker

vespa_docker_news = VespaDocker(port=8080)
app = vespa_docker_news.deploy_from_disk(
    application_name="news",
    application_folder="sample-apps/news/app-3-searching")

Waiting for configuration server.
Waiting for configuration server.
Waiting for application status.
Waiting for application status.
Finished deployment.


`app` can now be used to feed and query the application just deployed. Clean up after use:

In [7]:
vespa_docker_news.container.stop()
vespa_docker_news.container.remove()
rmtree("sample-apps", ignore_errors=True)